In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [ ]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [ ]:
# 获取高德地图API地址信息
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
#         print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('geocodes' in temp):
                re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                re_Dict_gaode["re_province_gaode"] = temp['geocodes'][0]['province']
                re_Dict_gaode["re_country_gaode"] = temp['geocodes'][0]['country']
                if ('citycode' in temp):
                    re_Dict_gaode["re_citycode_gaode"] = temp['geocodes'][0]['citycode']
                else:
                    pass
                re_Dict_gaode["re_city_gaode"] = temp['geocodes'][0]['city']
                re_Dict_gaode["re_district_gaode"] = temp['geocodes'][0]['district']
                re_Dict_gaode["re_level_gaode"] = temp['geocodes'][0]['level']
                re_Dict_gaode["re_township_gaode"] = temp['geocodes'][0]['township']
                re_Dict_gaode["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
                re_Dict_gaode["re_street_gaode"] = temp['geocodes'][0]['street']
                re_Dict_gaode["re_number_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [ ]:
# 解析百度地图API网址信息
def get_latlng_baidu(address,ak):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式
    ak = ak
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    return temp

In [ ]:
# 获取高德地图API地址信息
def get_receiver_lnglat_baidu(address):
    start_time = time.time()
    re_List_baidu = []
    for b in address:
#         print(b)
        re_Dict_baidu = {}
        try:
            temp = get_latlng_baidu(b,ak_baidu)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('result' in temp):
                re_Dict_baidu["re_Lng_baidu"] = temp['result']['location']['lng']
                re_Dict_baidu["re_Lat_baidu"] = temp['result']['location']['lat']
                re_Dict_baidu["re_Precise_baidu"] = temp['result']['precise']
                re_Dict_baidu["re_Confidence_baidu"] = temp['result']['confidence']
                re_Dict_baidu["re_Comprehension_baidu"] = temp['result']['comprehension']
                re_Dict_baidu["re_Level_baidu"] = temp['result']['level']
            else:
                pass
        re_List_baidu.append(re_Dict_baidu)
    #     time.sleep(1)
    end_time = time.time()
    print("百度总用时：", end_time - start_time)
    return re_List_baidu

In [ ]:
# ak码
ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'

In [ ]:
# 导入数据
inputpath = "H:\\work\\basic\\province\\DEC18\\"
inputFileName = "浙江(所在省).xlsx"
data = pd.read_excel(inputpath + inputFileName)


In [ ]:
data.columns 

In [ ]:
data["承运商"].value_counts()

In [ ]:
df = data[["所在省","所在市","收货地址","订单编号","承运商"]]

In [ ]:
df["address_all"]= df["所在省"] +"省"+ df["所在市"] + df["收货地址"]

In [ ]:
data.shape

In [ ]:
df.head()

In [ ]:
df_EMS = df[df["承运商"] == "EMS"]
df_JDWL = df[df["承运商"] == "京东物流"]
df_YD = df[df["承运商"] == "韵达"]
df_JDXX = df[df["承运商"] == "京东线下"]

In [ ]:
list_JDXX_gaode = get_receiver_lnglat_gaode(df_JDXX["address_all"])

In [ ]:
list_JDWL_gaode = get_receiver_lnglat_gaode(df_JDWL ["address_all"])

In [ ]:
list_YD_gaode = get_receiver_lnglat_gaode(df_YD ["address_all"])

In [ ]:
list_EMS_gaode = get_receiver_lnglat_gaode(df_EMS["address_all"])

In [ ]:
df_EMS_gaode = pd.DataFrame(list_EMS_gaode)
df_JDXX_gaode = pd.DataFrame(list_JDXX_gaode)
df_JDWL_gaode = pd.DataFrame(list_JDWL_gaode)
df_YD_gaode = pd.DataFrame(list_YD_gaode )

In [ ]:
# 导出数据
outpath = "D:\\work\\basic\\province\\"
name_ANSI = "1812_info_alter_lnglat1.xlsx"
# name_UTF8 = "1812_info_alter_lnglat.csv"

In [ ]:
print("验证完整收货地址：")
print(add[0:3])
print("&%"*50)